In [32]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from scipy import stats as st
import seaborn as sns

In [2]:
np.random.seed(12)
wisconsin_ages1 = stats.poisson.rvs(loc=18, mu=33, size=30)
wisconsin_ages2 = stats.poisson.rvs(loc=18, mu=13, size=20)
wisconsin_ages = np.concatenate((wisconsin_ages1, wisconsin_ages2))

print( wisconsin_ages.mean() )

42.8


In [4]:
np.random.seed(12)
minnesota_ages1 = stats.poisson.rvs(loc=18, mu=30, size=30)
minnesota_ages2 = stats.poisson.rvs(loc=18, mu=10, size=20)
minnesota_ages = np.concatenate((minnesota_ages1, minnesota_ages2))

In [33]:
st.ttest_ind(a= minnesota_ages,
                b= wisconsin_ages,
                equal_var=False)    # Assume samples have equal variance?

Ttest_indResult(statistic=-1.280392597784352, pvalue=0.20343036357319433)

In [7]:
df = pd.read_csv("gia_nha_dat_us.csv")
df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [8]:
df['MSZoning'].value_counts()

RL         1151
RM          218
FV           65
RH           16
C (all)      10
Name: MSZoning, dtype: int64

**BÀI TOÁN** So sánh giá trị trung bình của giá nhà thuộc các Zone khác nhau. Kiểm định xem giá bán trung bình của các Zone có bằng nhau hay không?

In [28]:
gr1 = df[df['MSZoning']=='RL'].sample(100)['SalePrice']
m1 = np.mean(gr1)
gr1.head()

931     117500
123     153900
338     202500
883     118500
1230    190000
Name: SalePrice, dtype: int64

In [29]:
gr2 = df[df['MSZoning']=='RM'].sample(100)['SalePrice']
m2 = np.mean(gr2)
gr2.head()

482    155000
821     93000
784    128000
29      68500
747    265979
Name: SalePrice, dtype: int64

Giả thiết thống kê:

**H0**: $\mu_1 = \mu_2$

**H1:** $\mu_1 \ne \mu_2$

In [34]:
def ztwo_samples(a,b,alternative='two-sided'):
    m1 = np.mean(a)
    n1 = len(a)
    if b is None:
        raise ValueError('Please insert 2nd sample!')
    n2 = len(b)
    m2 = np.mean(b)
    if min(n1,n2)<30:
        raise ValueError('All sample sides should be greater than 30!')
    v1 = np.var(a)
    v2 = np.var(b)
    se = np.sqrt(v1/n1+v2/n2)
    z_score = np.abs(m1-m2)/se
    if alternative in ['two-sided','0']:
        pvalue=st.norm.sf(np.abs(z_score))*2
    elif alternative in ['greater','1']:
        pvalue=st.norm.sf(z_score)
    elif alternative in ['less','-1']:
        pvalue=st.norm.cdf(z_score)
    else:
        raise ValueError('Alternative should be two-sided, greater or less!')
    return z_score, pvalue 

def results(p, vals=['mean1', 'mean2'], alpha=0.05):
    cols=['score', 'p_value', 'KetLuan']
    if p['p_value'] < alpha:
        p['KetLuan'] = f"Chấp nhận H1 với mức ý nghĩa {alpha}"
    if p['p_value'] >= alpha:
        p['KetLuan'] = f"Chấp nhận H0 với mức ý nghĩa {alpha}"
    df = pd.DataFrame(p, index=[''])
    if vals: cols = vals + cols
    return df[cols]

In [37]:
p = {}
p['mean1'] = np.mean(gr1)
p['mean2'] = np.mean(gr2)
p['score'], p['p_value'] = ztwo_samples(gr1, gr2)
print('Kiểm định Z test 2 sample')
results(p)

Kiểm định Z test 2 sample


,mean1,mean2,score,p_value,KetLuan
,190686.39,130486.23,5.996387,2.017553e-09,Chấp nhận H1 với mức ý nghĩa 0.05


Với t test 2samp

In [38]:
p = {}
p['mean1'] = np.mean(gr1)
p['mean2'] = np.mean(gr2)
p['score'], p['p_value'] = st.ttest_ind(gr1, gr2,equal_var=False)
print('Kiểm định t test 2 sample')
results(p)

Kiểm định t test 2 sample


,mean1,mean2,score,p_value,KetLuan
,190686.39,130486.23,5.96633,1.494801e-08,Chấp nhận H1 với mức ý nghĩa 0.05
